In [2]:
import json
from collections import defaultdict
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
lat_path = "/home/craig.car/summer2023/largefiles/lucretius/lucretius_lat_sents_labse.la.emb"
en_path = "/home/craig.car/summer2023/largefiles/lucretius/lucretius_en1893_sents_labse_NEW.en.emb"


In [6]:
dim = 768
src_embeddings = np.fromfile(lat_path, dtype=np.float32, count=-1)
tgt_embeddings = np.fromfile(en_path, dtype=np.float32, count=-1)
src_embeddings.resize(src_embeddings.shape[0] // dim, dim)
tgt_embeddings.resize(tgt_embeddings.shape[0] // dim, dim)

In [7]:
def compute_sim_matrix(src_embeds, tgt_embeds):
    # using cosine sim
    return cosine_similarity(src_embeds, tgt_embeds)

In [8]:
# compute similarity matrix
sim_matrix = compute_sim_matrix(src_embeddings, tgt_embeddings)
sim_matrix.shape

(2428, 14648)

In [10]:
lat_dict_path = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lat_sent2book_dict.json"

with open(lat_dict_path) as f:
    lat_sent_2_section_name = json.load(f)

en_dict_path = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/en1893_sent2section_dict_jul25.json"
with open(en_dict_path) as f_:
    en_sent_2_section_name = json.load(f_)


In [17]:
#TEST

In [18]:
sorted_0 = np.flip(np.sort(sim_matrix[51]))
top_10_0 = sorted_0[:10]


In [19]:
sorted_0[:10]

array([0.7112274 , 0.54273117, 0.53878355, 0.5227177 , 0.5172635 ,
       0.5124127 , 0.5109819 , 0.49658653, 0.48991406, 0.48897603],
      dtype=float32)

In [20]:
top_10_dict = defaultdict(list)
for rank in range(0,10):
    for idx_tgt_sent, score in enumerate(sim_matrix[51]):
        if score == top_10_0[rank]:
            top_10_dict[rank].append(idx_tgt_sent)

In [21]:
top_10_dict

defaultdict(list,
            {0: [840],
             1: [6775],
             2: [7542],
             3: [241],
             4: [7447],
             5: [6147],
             6: [12555],
             7: [2249],
             8: [896],
             9: [6755]})

In [22]:
src_chapter = lat_sent_2_section_name[str(0)]


In [35]:
top_1_chap_tgt = en_sent_2_section_name[str(top_10_dict[0][0])]

In [36]:
top_1_chap_tgt = top_1_chap_tgt[-5:]
top_1_chap_tgt

'book1'

In [37]:
top_10_chaps = []
for rank in range(0,10):
    tgt_idx = top_10_dict[rank]
    tgt_chaps = []
    for idx in tgt_idx:
        tgt_chaps.append(en_sent_2_section_name[str(idx)][-5:])
    top_10_chaps.extend(tgt_chaps)
if len(top_10_chaps) != 10:
    print(rank)


In [38]:
top_10_chaps

['book1',
 'book5',
 'book6',
 'book0',
 'book6',
 'book5',
 'book5',
 'book2',
 'book1',
 'book5']

In [39]:
tp = 0
recall_top_1 = 0
recall_top_10 = 0
if top_10_chaps[0] == src_chapter:
    recall_top_1 += 1
for chapter in top_10_chaps:
    if chapter == src_chapter:
        tp += 1
recall_top_10 += tp/10

In [40]:
tp

2

In [41]:
recall_top_1

1

In [42]:
recall_top_10

0.2

In [62]:
num_top_1 = 0
num_top_10 = 0
score_top_10 = 0

# for idx_vector, vector in enumerate(sim_matrix[:10]):
for idx_vector, vector in enumerate(sim_matrix):
    tp = 0
    # sort vector and get top 10 scores
    sorted_sim_vec = np.flip(np.sort(vector))
    top_10_scores = sorted_sim_vec[:10]
    # print(f"top 10 scores: {sorted_sim_vec[0:10]}")
    
    # get idx of tgt sents with top 10 scores
    top_10_dict = defaultdict(list)
    for rank in range(0,10):
        for idx_tgt_sent, score in enumerate(vector):
            if score == top_10_scores[rank]:
                top_10_dict[rank].append(idx_tgt_sent)
    # print(top_10_dict)
    
    # get src sent chapter and compare to predicted tgt sents
    src_chapter = lat_sent_2_section_name[str(idx_vector)]
    top_10_chaps = []
    for rank in range(0,10):
        tgt_idx = top_10_dict[rank]
        tgt_chaps = []
        for idx in tgt_idx:
            tgt_chaps.append(en_sent_2_section_name[str(idx)][-5:])
        top_10_chaps.extend(tgt_chaps)
    if len(top_10_chaps) != 10:
        print(f"len top_10_chaps longer than 10 en {idx_vector}")
    # print(src_chapter)
    # print(top_10_chaps)
    # compare top scores' chapters to scr sent chap
    
    # # compare the sets: if text, then last 5 characters in format "book[1-6]"
    # src_chaps_compare = set()
    # for src_chap in src_chapter:
    #     src_chaps_compare.add(src_chap[-5:])
    # tgt_chaps_compare = set()
    # for tgt_chap in top_10_chaps:
    #     tgt_chaps_compare.add(tgt_chap[-5:])
    # # print(f"{src_chaps_compare} : {tgt_chaps_compare}")
    
    top_1_score_idx = top_10_dict[0][0]
    top_1_score_chap = en_sent_2_section_name[str(top_1_score_idx)][-5:]
    # print(top_1_score_chap)
    # if top_10_chaps[0] == src_chapter:
    if top_1_score_chap == src_chapter:
        num_top_1 += 1
        print(top_1_score_idx)
    for chapter in top_10_chaps:
        if chapter == src_chapter:
            tp += 1
    score_top_10 += tp/len(top_10_chaps)
    if tp != 0:
        num_top_10 += 1
    
    print(f"processed vector {idx_vector}")

483
processed vector 0
532
processed vector 1
533
processed vector 2
534
processed vector 3
535
processed vector 4
536
processed vector 5
537
processed vector 6
processed vector 7
575
processed vector 8
577
processed vector 9
613
processed vector 10
614
processed vector 11
processed vector 12
658
processed vector 13
processed vector 14
617
processed vector 15
659
processed vector 16
692
processed vector 17
693
processed vector 18
697
processed vector 19
698
processed vector 20
699
processed vector 21
700
processed vector 22
701
processed vector 23
721
processed vector 24
724
processed vector 25
725
processed vector 26
processed vector 27
728
processed vector 28
730
processed vector 29
731
processed vector 30
733
processed vector 31
753
processed vector 32
754
processed vector 33
755
processed vector 34
757
processed vector 35
758
processed vector 36
processed vector 37
779
processed vector 38
780
processed vector 39
782
processed vector 40
783
processed vector 41
819
processed vector 4

In [57]:
len(sim_matrix)

2428

In [58]:
num_top_1/len(sim_matrix)

0.7652388797364086

In [59]:
num_top_10/len(sim_matrix)

0.9724052718286655

In [60]:
score_top_10/len(sim_matrix)

0.3562103739204239